In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time, os
from tqdm import tqdm
import gc

np.seterr(divide='ignore', invalid='ignore')

objc[4416]: Class CaptureDelegate is implemented in both /Users/user/miniforge3/envs/tensorflow/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x11fa06840) and /Users/user/miniforge3/envs/tensorflow/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x121f7c860). One of the two will be used. Which one is undefined.
objc[4416]: Class CVWindow is implemented in both /Users/user/miniforge3/envs/tensorflow/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x11fa06890) and /Users/user/miniforge3/envs/tensorflow/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x120598a68). One of the two will be used. Which one is undefined.
objc[4416]: Class CVView is implemented in both /Users/user/miniforge3/envs/tensorflow/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x11fa068b8) and /Users/user/miniforge3/envs/tensorflow/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x120598a90). One of the two will be used. Which one is u

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
import pandas as pd
trainpath = "/Users/user/PycharmProjects/DL_hands/open/train/"
df = pd.read_csv("/Users/user/PycharmProjects/DL_hands/open/train.csv")

In [3]:
def fileload(num):
    path = trainpath
    filename = ""
    if num<10:
        filename = "TRAIN_"+"00"+str(num)+".mp4"
    elif num<100:
        filename = "TRAIN_"+"0"+ str(num)+ ".mp4"
    else:
        filename = "TRAIN_"+ str(num)+ ".mp4"

    cap = cv2.VideoCapture(path + filename)
    #width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    #height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    #fps = cap.get(cv2.CAP_PROP_FPS)
    
    return cap

In [4]:
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1,
                      min_detection_confidence=0.5,
                      min_tracking_confidence=0.5)
mpDraw = mp.solutions.drawing_utils

In [5]:
cd /Users/user/PycharmProjects/DL_hands/open/train

/Users/user/PycharmProjects/DL_hands/open/train


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [6]:
def play(cap):
    data = np.array([])
    found = 0

    while cap.isOpened():
        ret, img = cap.read()

        if ret:
            
            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = hands.process(imgRGB)
            h, w, c = img.shape

            ###### Data 
            if results.multi_hand_landmarks:
                found += 1
                for res in results.multi_hand_landmarks:
                    mpDraw.draw_landmarks(img, res, mpHands.HAND_CONNECTIONS)
                    joint = np.zeros((21, 3))
                    for id, lm in enumerate(res.landmark):
                        joint[id] = [lm.x, lm.y, lm.z]

                    # Compute Angles (1) (2) (3) Fingers
                    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                    v = v2 - v1

                    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                    angle = np.arccos(np.einsum('nt,nt->n',
                        v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                        v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # (15,0)
                    
                    # rad 2 deg
                    angle = np.degrees(angle)
                    angle = np.array(angle, dtype=np.int32)

                    tmp = np.array([angle])
                    tmp = tmp.flatten()
                    data = np.append(data, tmp)
            else:
                tmp = np.zeros(15)
                tmp[:] = np.NaN
                data = np.append(data, tmp)
                    
        else:
            cap.release()
            break

        #cv2.imshow("Image", img)
        #cv2.waitKey(1)
        #cv2.destroyAllWindows()
    return np.array(data, dtype=np.int32), found

In [7]:
#filetotal = len(os.listdir(trainpath))
filetotal = 610

In [8]:
df['found'] = np.zeros(filetotal)

In [9]:
def npsave(n):
    tmp, found = play(fileload(n))
    df['found'][n] = int(found)
    if (len(tmp) > 0):
        if n<10:
            with open('/Users/user/PycharmProjects/DL_hands/open/data3/fingers_info00'+str(n), "wb") as f:
                np.save(f, tmp)
        elif n<100:
            with open('/Users/user/PycharmProjects/DL_hands/open/data3/fingers_info0'+str(n), "wb") as f:
                np.save(f, tmp)
        else:
            with open('/Users/user/PycharmProjects/DL_hands/open/data3/fingers_info'+str(n), "wb") as f:
                np.save(f, tmp)
        return None
    else:
        return n

In [10]:
for i in tqdm(range(filetotal)):
    num = npsave(i)

  0%|                                                   | 0/610 [00:00<?, ?it/s]/var/folders/np/rt2hjxz968lf34_lfd897zp00000gn/T/ipykernel_4416/4180892090.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['found'][n] = int(found)
100%|█████████████████████████████████████████| 610/610 [04:00<00:00,  2.54it/s]


In [11]:
df

,id,path,label,found
0,TRAIN_000,./train/TRAIN_000.mp4,3,30.0
1,TRAIN_001,./train/TRAIN_001.mp4,0,29.0
2,TRAIN_002,./train/TRAIN_002.mp4,1,30.0
3,TRAIN_003,./train/TRAIN_003.mp4,4,29.0
4,TRAIN_004,./train/TRAIN_004.mp4,4,29.0
...,...,...,...,...
605,TRAIN_605,./train/TRAIN_605.mp4,0,29.0
606,TRAIN_606,./train/TRAIN_606.mp4,2,26.0
607,TRAIN_607,./train/TRAIN_607.mp4,1,30.0
608,TRAIN_608,./train/TRAIN_608.mp4,4,29.0


In [26]:
failed_clips = (df['found'] == 0)
df[failed_clips]

failed_clips = []
for i in range(filetotal):
    if df['found'][i] == 0:
        failed_clips.append(i)

In [27]:

np.array(failed_clips)
#failed_clips = [  8,  74,  82, 126, 149, 211, 213, 278, 284, 312, 386, 457, 564]

array([  8,  74,  82, 126, 149, 211, 213, 278, 284, 312, 386, 457, 564])

In [13]:
"""
for i in failed_clips:
    if npsave(i) == None:
        failed_clips.remove(i)
        print(i, "success")
    else:
        continue
"""

'\nfor i in failed_clips:\n    if npsave(i) == None:\n        failed_clips.remove(i)\n        print(i, "success")\n    else:\n        continue\n'

In [15]:
def saturate_contrast(p, num):
    # num eg) 1.xx 
    pic = p.copy()
    pic = pic.astype('int64')
    pic = np.clip(pic*num, 0, 255)
    pic = pic.astype('uint8')
    return pic

In [28]:
np.array(failed_clips)

array([  8,  74,  82, 126, 149, 211, 213, 278, 284, 312, 386, 457, 564])

In [29]:
def gamma(img, num):
    g = num
    #g = float(input("감마 값 : "))
    out = img.copy()
    #out = img.astype(np.float)
    out = ((out / 255) ** (1 / g)) * 255
    out = out.astype(np.uint8)
    return out

In [30]:
def play_bright(cap, g):
    data = np.array([])
    found = 0
            
    while cap.isOpened():
        ret, img = cap.read()

        if ret:
            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            imgRGB = gamma(imgRGB, g)
            #imgRGB = saturate_contrast(imgRGB, 1.1)
            results = hands.process(imgRGB)

            ###### Data 
            if results.multi_hand_landmarks:
                found += 1
                for res in results.multi_hand_landmarks:
                    mpDraw.draw_landmarks(img, res, mpHands.HAND_CONNECTIONS)
                    joint = np.zeros((21, 3))
                    for id, lm in enumerate(res.landmark):
                        joint[id] = [lm.x, lm.y, lm.z]

                    # Compute Angles (1) (2) (3) Fingers
                    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                    v = v2 - v1

                    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                    angle = np.arccos(np.einsum('nt,nt->n',
                        v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                        v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # (15,0)
                    
                    # rad 2 deg
                    angle = np.degrees(angle)
                    angle = np.array(angle, dtype=np.int32)

                    tmp = np.array([angle])
                    tmp = tmp.flatten()
                    data = np.append(data, tmp)
            else:
                tmp = np.zeros(15)
                tmp[:] = np.NaN
                data = np.append(data, tmp)
                    
        else:
            cap.release()
            break
        img2 = cv2.cvtColor(imgRGB, cv2.COLOR_BGR2RGB)
        cv2.imshow("Image", img2)
        cv2.waitKey(1)
        cv2.destroyAllWindows()
    return np.array(data, dtype=np.int32), found

In [31]:
def npsave_bright(n, g):
    cp = fileload(n)
    tmp, found = play_bright(cp, g)
    df['found'][n] = int(found)
    
    if (len(tmp) > 0):
        if n<10:
            with open('/Users/user/PycharmProjects/DL_hands/open/data3/fingers_info00'+str(n), "wb") as f:
                np.save(f, tmp)
        elif n<100:
            with open('/Users/user/PycharmProjects/DL_hands/open/data3/fingers_info0'+str(n), "wb") as f:
                np.save(f, tmp)
        else:
            with open('/Users/user/PycharmProjects/DL_hands/open/data3/fingers_info'+str(n), "wb") as f:
                np.save(f, tmp)
        return None
    else:
        return n

In [32]:
for i in failed_clips:
    for j in range(8, 20):
        num = npsave_bright(i, j/10)

        if num == None:
            failed_clips.remove(i)
            break


/var/folders/np/rt2hjxz968lf34_lfd897zp00000gn/T/ipykernel_4416/2068204905.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['found'][n] = int(found)


In [33]:
failed_clips


[74, 126, 211, 278, 312, 457]

In [34]:
df.to_pickle("/Users/user/PycharmProjects/DL_hands/open/df.pkl")